<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/fraud/response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import json as js
import re

In [3]:
hospital = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/health_office.csv')
hospital.columns

<ipython-input-3-d69fde0a6e6e>:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  hospital = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/health_office.csv')


Index(['ชื่อ', 'รหัส 9 หลักใหม่', 'รหัส 9 หลัก', 'รหัส 5 หลัก',
       'เลขอนุญาตให้ประกอบสถานบริการสุขภาพ 11 หลัก', 'ประเภทองค์กร',
       'ประเภทหน่วยบริการสุขภาพ', 'สังกัด', 'แผนก/กรม', 'ระดับโรงพยาบาล',
       'เตียงที่ใช้จริง', 'สถานะการใช้งาน', 'เขตบริการ', 'ที่อยู่',
       'รหัสจังหวัด', 'จังหวัด', 'รหัสอำเภอ', 'อำเภอ/เขต', 'รหัสตำบล',
       'ตำบล/แขวง', 'หมู่', 'รหัสไปรษณีย์', 'แม่ข่าย', 'วันที่ก่อตั้ง',
       'วันที่ปิดบริการ', 'อัพเดตล่าสุด(เริ่ม 05/09/2566)'],
      dtype='object')

In [4]:
data = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/opd_discharge_August_2024.csv')
cols_to_drop = [col for col in data.columns if col.endswith('_txt')]
data = data.drop(columns=cols_to_drop)
df = data[data['claim_status'].isin(['Approve', 'Decline'])]
print(df['claim_status'].value_counts())

claim_status
Approve    15581
Decline      184
Name: count, dtype: int64


In [5]:
df['hospital_code'] = df['hospital_code'].astype(str)
hospital['hospital_code'] = hospital['รหัส 5 หลัก'].str.replace('="', '').str.replace('"', '')
hospital = hospital[['hospital_code', 'ชื่อ']].rename(columns={'ชื่อ': 'hospital_name'})
df = df.merge(hospital, on='hospital_code', how='left')

<ipython-input-5-83a06c097e2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hospital_code'] = df['hospital_code'].astype(str)


In [6]:
hospital[hospital['hospital_code'] == '40765']

,hospital_code,hospital_name
1394,40765,โรงพยาบาลทั่วไปขนาดใหญ่เกษมราษฎร์ รัตนาธิเบศร์


In [7]:
focus = df[df['order_item'] != '[]'][['hospital_code', 'hospital_name', 'cid', 'insurance', 'claim_status','total_bill_amount', 'diagnosis', 'billing', 'order_item', 'response']]

In [8]:
decline = focus[focus['claim_status'].astype(str)=='Decline']
decline['insurance'].value_counts()

,count
insurance,
tli,23
aia,19
mtl,12
fwd,3


## **❗case: decline**
- **tli**: เข้าข้อยกเว้น, ICD10 อยู่ในกลุ่มโรค

- **aia**: ไม่สามารถเคลมได้ เนื่องจากสัญญาเพิ่มเติมสิ้นผลบังคับ, เบิกซ้ำซ้อนมากกว่า 1 ครั้ง/วัน, ต้องการข้อมูลทางการแพทย์เพิ่มเติม, ผลประโยชน์คุ้มครองเฉพาะค่ารักษาพยาบาลกรณีอุบัติเหตุ

- **mtl**: กรุณาสำรองจ่ายและส่งเอกสารการเคลมให้กับบริษัทฯโดยตรง, โรคมีระยะเวลาการรอคอย 180 วัน, ส่ง fax claim, เงื่อนไขโรคยกเว้นเฉพาะรายบุคคล, โรคที่เข้าข่ายในเงื่อนไขข้อยกเว้นกรมธรรม์สัญญาสุขภาพ

- **fwd**: copay == ผู้ป่วยต้องจ่ายเอง

In [28]:
js.loads(decline[decline['insurance']=='aia']['response'].iloc[2])

{'detail': [],
 'claim_no': 'C029086468',
 'insurer_detail': [{'claim_no': 'C029086468',
   'non_cover': '',
   'benefit_code': 'HSBypass',
   'copay_amount': '',
   'coverage_code': 'H&S Extra (new standard) ไม่สามารถใช้บริการเรียกร้องสินไหมได้เนื่องจากจำเป็นต้องพิจารณาข้อมูลทางการแพทย์เพิ่มเติม หากต้องการใช้สิทธิ์ กรุณาตรวจสอบความคุ้มครองโดยเจ้าหน้าที่ AIA ( เวลาทำการ 08.30 - 19.00 น.)',
   'excess_amount': '',
   'policy_number': 'T232375422',
   'insurance_type': '',
   'order_sequence': '1',
   'trailer_number': '',
   'approved_amount': '',
   'result_description': 'กรุณาตรวจสอบความคุ้มครองที่เมนู Claim status เวลาทำการ 08.30 – 19.00'}],
 'transaction_no': '3cf7edde-7a78-4ecc-b1f5-aa57b6889514',
 'final_pss_score': '0',
 'transaction_uid': '865787db-8337-47fb-9a9d-4e1b4b94050d',
 'total_excess_amount': '1486.70',
 'total_approved_amount': '0.00'}

In [ ]:
focus[focus['cid']=='1103800114329']['order_item'].iloc[0]

In [10]:
approve = focus[focus['claim_status'].astype(str)=='Approve']
approve['insurance'].value_counts()

,count
insurance,
mtl,2068
aia,1572
fwd,868
tli,265


เพื่อค้นหา `billing` ที่เป็นรายการที่พบร่วมกันบ่อยในผู้ป่วยที่เป็นโรคเดียวกัน **และหาค่าราคากลาง**ของ `billing` นั้น

In [12]:
import json

# แปลง JSON string ให้เป็น dictionary หรือ list
focus['diagnosis'] = focus['diagnosis'].apply(lambda x: json.loads(x))
focus['order_item'] = focus['order_item'].apply(lambda x: json.loads(x))

In [49]:
import json

# กรองข้อมูลตามการวินิจฉัยที่ต้องการ เช่น "J069"
filtered_data = focus[
    focus['diagnosis'].apply(lambda x: any(d.get('icd10') == 'K300' for d in x))
]

# ตรวจสอบและโหลดข้อมูล billing
billing_items = []
for items in filtered_data['billing']:
    items = json.loads(items)  # แปลง JSON string เป็น list ของ dictionary
    for item in items:
        billing_items.append(item)

# แปลง billing_items เป็น DataFrame เพื่อช่วยในการคำนวณ
billing_items_df = pd.DataFrame(billing_items)

# แปลง billing_initial เป็น float ใน billing_items_df
billing_items_df['billing_initial'] = billing_items_df['billing_initial'].astype(float)

# คำนวณค่าราคากลาง (mean) ของแต่ละ simb_billing_code, รวม local_billing_name และจำนวนการสั่ง (count)
billing_summary = billing_items_df.groupby(['simb_billing_code', 'local_billing_name']).agg(
    average_initial_price=('billing_initial', 'mean'),
    billing_count=('billing_initial', 'count')
).reset_index()

display(billing_summary)

,simb_billing_code,local_billing_name,average_initial_price,billing_count
0,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,1050.875000,8
1,1.1.12,Nursing and Midwifery Charge ค่าบริการพยาบาล,211.875000,8
2,1.1.14(1),Packaged Medical Charges ค่าบริการชุดเหมาจ่าย,100.000000,1
3,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,181.875000,8
4,1.1.2(1),Medical Supplies 1 เวชภัณฑ์1,146.666667,3
5,1.2.1,Physician Evaluation and Management Services,700.000000,2
6,1.2.1(1),First Outpatient care before admission ค่าแพท...,511.666667,6


In [48]:
import json

# กรองข้อมูลตามการวินิจฉัยที่ต้องการ เช่น "J069"
filtered_data = focus[
    focus['diagnosis'].apply(lambda x: any(d.get('icd10') == 'J069' for d in x))
]

# ตรวจสอบและโหลดข้อมูล billing
billing_items = []
for items in filtered_data['billing']:
    items = json.loads(items)  # แปลง JSON string เป็น list ของ dictionary
    for item in items:
        billing_items.append(item)

# แปลง billing_items เป็น DataFrame เพื่อช่วยในการคำนวณ
billing_items_df = pd.DataFrame(billing_items)

# แปลง billing_initial เป็น float ใน billing_items_df
billing_items_df['billing_initial'] = billing_items_df['billing_initial'].astype(float)

# คำนวณค่าราคากลาง (mean) ของแต่ละ simb_billing_code, รวม local_billing_name และจำนวนการสั่ง (count)
billing_summary = billing_items_df.groupby(['simb_billing_code', 'local_billing_name']).agg(
    average_initial_price=('billing_initial', 'mean'),
    billing_count=('billing_initial', 'count')
).reset_index()

display(billing_summary)

,simb_billing_code,local_billing_name,average_initial_price,billing_count
0,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,929.654618,249
1,1.1.10,Physical Therapy ค่ากายภาพบำบัด,990.000000,1
2,1.1.12,Nursing and Midwifery Charge ค่าบริการพยาบาล,149.814050,242
3,1.1.14(1),Packaged Medical Charges ค่าบริการชุดเหมาจ่าย,128.000000,5
4,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,167.717213,244
5,1.1.2(1),Medical Supplies 1 เวชภัณฑ์1,76.900000,30
6,1.1.4,Laboratory Investigation and Pathology ค่าตรว...,976.088235,34
7,1.1.5(1),Standard Diagnostic ค่าตรวจรังสีปกติ รังสี...,550.000000,2
8,1.1.6,Special Diagnostic ค่าตรวจวินิจฉัยโดยวิธีพิเ...,2150.000000,2
9,1.1.7(1),General Medical Equipment ค่าอุปกรณ์นอกห้องผ่...,412.500000,4


In [50]:
import json

# กรองข้อมูลตามการวินิจฉัยที่ต้องการ
filtered_data = focus[
    focus['diagnosis'].apply(lambda x: any(d.get('icd10') == 'J029' for d in x))
]

# ตรวจสอบและโหลดข้อมูล billing
billing_items = []
for items in filtered_data['billing']:
    items = json.loads(items)  # แปลง JSON string เป็น list ของ dictionary
    for item in items:
        billing_items.append(item)

# แปลง billing_items เป็น DataFrame เพื่อช่วยในการคำนวณ
billing_items_df = pd.DataFrame(billing_items)

# แปลง billing_initial เป็น float ใน billing_items_df
billing_items_df['billing_initial'] = billing_items_df['billing_initial'].astype(float)

# คำนวณค่าราคากลาง (mean) ของแต่ละ simb_billing_code, รวม local_billing_name และจำนวนการสั่ง (count)
billing_summary = billing_items_df.groupby(['simb_billing_code', 'local_billing_name']).agg(
    average_initial_price=('billing_initial', 'mean'),
    billing_count=('billing_initial', 'count')
).reset_index()

display(billing_summary)

,simb_billing_code,local_billing_name,average_initial_price,billing_count
0,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,956.404494,178
1,1.1.12,Nursing and Midwifery Charge ค่าบริการพยาบาล,143.128492,179
2,1.1.14(1),Packaged Medical Charges ค่าบริการชุดเหมาจ่าย,230.000000,3
3,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,133.784530,181
4,1.1.2(1),Medical Supplies 1 เวชภัณฑ์1,66.540541,37
5,1.1.4,Laboratory Investigation and Pathology ค่าตรว...,881.538462,13
6,1.1.5(1),Standard Diagnostic ค่าตรวจรังสีปกติ รังสี...,491.250000,8
7,1.1.6,Special Diagnostic ค่าตรวจวินิจฉัยโดยวิธีพิเ...,400.000000,1
8,1.1.7(1),General Medical Equipment ค่าอุปกรณ์นอกห้องผ่...,242.090909,11
9,1.2.1,Physician Evaluation and Management Services,539.310345,29


In [51]:
import json

# กรองข้อมูลตามการวินิจฉัยที่ต้องการ
filtered_data = focus[
    focus['diagnosis'].apply(lambda x: any(d.get('icd10') == 'J000' for d in x))
]

# ตรวจสอบและโหลดข้อมูล billing
billing_items = []
for items in filtered_data['billing']:
    items = json.loads(items)  # แปลง JSON string เป็น list ของ dictionary
    for item in items:
        billing_items.append(item)

# แปลง billing_items เป็น DataFrame เพื่อช่วยในการคำนวณ
billing_items_df = pd.DataFrame(billing_items)

# แปลง billing_initial เป็น float ใน billing_items_df
billing_items_df['billing_initial'] = billing_items_df['billing_initial'].astype(float)

# คำนวณค่าราคากลาง (mean) ของแต่ละ simb_billing_code, รวม local_billing_name และจำนวนการสั่ง (count)
billing_summary = billing_items_df.groupby(['simb_billing_code', 'local_billing_name']).agg(
    average_initial_price=('billing_initial', 'mean'),
    billing_count=('billing_initial', 'count')
).reset_index()

display(billing_summary)

,simb_billing_code,local_billing_name,average_initial_price,billing_count
0,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,1006.267241,116
1,1.1.12,Nursing and Midwifery Charge ค่าบริการพยาบาล,146.902655,113
2,1.1.14(1),Packaged Medical Charges ค่าบริการชุดเหมาจ่าย,300.000000,1
3,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,144.956522,115
4,1.1.2(1),Medical Supplies 1 เวชภัณฑ์1,79.571429,35
5,1.1.4,Laboratory Investigation and Pathology ค่าตรว...,923.750000,12
6,1.1.5(1),Standard Diagnostic ค่าตรวจรังสีปกติ รังสี...,708.333333,6
7,1.1.7(1),General Medical Equipment ค่าอุปกรณ์นอกห้องผ่...,151.333333,9
8,1.2.1,Physician Evaluation and Management Services,552.840909,66
9,1.2.1(1),First Outpatient care before admission ค่าแพท...,522.000000,50


## final

In [53]:
import json
import pandas as pd

# สร้าง DataFrame เปล่าเพื่อเก็บผลลัพธ์
all_billing_summary = []

# กรองข้อมูลตามแต่ละโรคที่ไม่ซ้ำกันใน 'diagnosis'
unique_icd10_codes = focus['diagnosis'].apply(lambda x: [d.get('icd10') for d in x]).explode().unique()

for icd10 in unique_icd10_codes:
    # กรองข้อมูลตามโรคที่กำหนด
    filtered_data = focus[
        focus['diagnosis'].apply(lambda x: any(d.get('icd10') == icd10 for d in x))
    ]

    # ตรวจสอบและโหลดข้อมูล billing
    billing_items = []
    for items in filtered_data['billing']:
        # แปลง JSON string เป็น list ของ dictionary
        items = json.loads(items)  # ปรับให้เป็น JSON string
        for item in items:
            billing_items.append(item)

    # แปลง billing_items เป็น DataFrame เพื่อช่วยในการคำนวณ
    billing_items_df = pd.DataFrame(billing_items)

    # แปลง billing_initial เป็น float ใน billing_items_df
    billing_items_df['billing_initial'] = billing_items_df['billing_initial'].astype(float)

    # คำนวณค่าราคากลาง (mean) ของแต่ละ simb_billing_code, รวม local_billing_name และจำนวนการสั่ง (count)
    billing_summary = billing_items_df.groupby(['simb_billing_code', 'local_billing_name']).agg(
        average_initial_price=('billing_initial', 'mean'),
        billing_count=('billing_initial', 'count')
    ).reset_index()

    # เพิ่มคอลัมน์ icd10 เพื่อเก็บรหัสโรค
    billing_summary['icd10'] = icd10

    # เก็บผลลัพธ์ลงใน all_billing_summary
    all_billing_summary.append(billing_summary)

# รวมผลลัพธ์ทั้งหมดเข้าด้วยกัน
final_summary = pd.concat(all_billing_summary, ignore_index=True)

# แสดงผลรายการ
display(final_summary)

,simb_billing_code,local_billing_name,average_initial_price,billing_count,icd10
0,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,1083.687500,16,S711
1,1.1.12,Nursing and Midwifery Charge ค่าบริการพยาบาล,310.666667,15,S711
2,1.1.14(1),Packaged Medical Charges ค่าบริการชุดเหมาจ่าย,214.285714,7,S711
3,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,213.333333,15,S711
4,1.1.2(1),Medical Supplies 1 เวชภัณฑ์1,464.133333,15,S711
...,...,...,...,...,...
7515,1.1.14(1),Packaged Medical Charges ค่าบริการชุดเหมาจ่าย,150.000000,1,S223
7516,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,100.000000,1,S223
7517,1.1.2(1),Medical Supplies 1 เวชภัณฑ์1,516.000000,1,S223
7518,1.1.5(1),Standard Diagnostic ค่าตรวจรังสีปกติ รังสี...,900.000000,1,S223


In [107]:
import json

# กรองข้อมูลตามการวินิจฉัยที่ต้องการ
filtered_data = focus[
    focus['diagnosis'].apply(lambda x: any(d.get('icd10') == 'B974' for d in x))
]

# ตรวจสอบและโหลดข้อมูล billing
billing_items = []
for items in filtered_data['billing']:
    items = json.loads(items)  # แปลง JSON string เป็น list ของ dictionary
    for item in items:
        billing_items.append(item)

# แปลง billing_items เป็น DataFrame เพื่อช่วยในการคำนวณ
billing_items_df = pd.DataFrame(billing_items)

# แปลง billing_initial เป็น float ใน billing_items_df
billing_items_df['billing_initial'] = billing_items_df['billing_initial'].astype(float)

# คำนวณค่าราคากลาง (mean) ของแต่ละ simb_billing_code, รวม local_billing_name และจำนวนการสั่ง (count)
billing_summary = billing_items_df.groupby(['simb_billing_code', 'local_billing_name']).agg(
    average_initial_price=('billing_initial', 'mean'),
    billing_count=('billing_initial', 'count')
).reset_index()

display(billing_summary)
# Function to calculate the total average initial price
def calculate_total_average(billing_summary_df):
    # Calculate the total average price
    total_average_price = billing_summary_df['average_initial_price'].sum()
    return total_average_price

# Call the function and display the total average initial price
total_average_initial_price = calculate_total_average(billing_summary)
print("Total Average Initial Price:", total_average_initial_price)

,simb_billing_code,local_billing_name,average_initial_price,billing_count
0,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,400.846154,13
1,1.1.10,Physical Therapy ค่ากายภาพบำบัด,280.000000,1
2,1.1.12,Nursing and Midwifery Charge ค่าบริการพยาบาล,161.923077,13
3,1.1.14(1),Packaged Medical Charges ค่าบริการชุดเหมาจ่าย,290.000000,3
4,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,171.538462,13
5,1.1.2(1),Medical Supplies 1 เวชภัณฑ์1,36.666667,9
6,1.1.4,Laboratory Investigation and Pathology ค่าตรว...,640.000000,2
7,1.2.1(1),First Outpatient care before admission ค่าแพท...,469.230769,13


Total Average Initial Price: 2450.205128205128


In [111]:
# กรองข้อมูลตามการวินิจฉัยที่ต้องการ
filtered_data = focus[
    focus['diagnosis'].apply(lambda x: any(d.get('icd10') == 'B974' for d in x))
]
js.loads(filtered_data['billing'].iloc[0])

[{'billing_initial': '450',
  'billing_discount': '45',
  'simb_billing_code': '1.1.1(3)',
  'billing_net_amount': '405.00',
  'local_billing_code': '50',
  'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'},
 {'billing_initial': '45',
  'billing_discount': '0',
  'simb_billing_code': '1.1.12',
  'billing_net_amount': '45.00',
  'local_billing_code': '240',
  'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'},
 {'billing_initial': '50',
  'billing_discount': '0',
  'simb_billing_code': '1.1.14(2)',
  'billing_net_amount': '50.00',
  'local_billing_code': '280',
  'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'},
 {'billing_initial': '500',
  'billing_discount': '0',
  'simb_billing_code': '1.2.1(1)',
  'billing_net_amount': '500.00',
  'local_billing_code': '310',
  'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}]

In [67]:
print(final_summary['simb_billing_code'].nunique())
print(final_summary['icd10'].nunique())
unique_icd10_codes

27
1191


array(['S711', 'W220', 'J300', ..., 'I676', 'U099', 'S223'], dtype=object)

In [64]:
final_summary[final_summary['local_billing_name']=='Outpatient Medication   ค่ายาผู้ป่วยนอก']

,simb_billing_code,local_billing_name,average_initial_price,billing_count,icd10
0,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,1083.687500,16,S711
11,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,1451.500000,60,W220
25,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,632.200000,5,J300
33,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,956.404494,178,J029
47,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,614.000000,2,N944
...,...,...,...,...,...
7497,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,747.000000,1,J09
7500,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,1188.000000,1,I458
7505,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,800.000000,1,I676
7509,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,690.000000,1,U099


In [66]:
final_summary[final_summary['local_billing_name']=='Other Medical Service Charge ค่าบริการทางการแพทย์']

,simb_billing_code,local_billing_name,average_initial_price,billing_count,icd10
3,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,213.333333,15,S711
15,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,191.776316,76,W220
27,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,168.000000,5,J300
36,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,133.784530,181,J029
49,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,167.500000,2,N944
...,...,...,...,...,...
7499,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,50.000000,1,J09
7502,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,50.000000,1,I458
7507,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,50.000000,1,I676
7511,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,50.000000,1,U099


In [65]:
print(final_summary['local_billing_name'].iloc[7516])

Other Medical Service Charge ค่าบริการทางการแพทย์
